## Importing mip

In [15]:
from mip import Model, xsum, minimize, BINARY
from itertools import product

## Define the array of cities

In [16]:
places = [
    "Antwerp",
    "Bruges",
    "C-Mine",
    "Dinant",
    "Ghent",
    "Grand-Place de Bruxelles",
    "Hasselt",
    "Leuven",
    "Mechelen",
    "Mons",
    "Montagne de Bueren",
    "Namur",
    "Remouchamps",
    "Waterloo",
]

## Define the distance matrix for TSP

In [17]:
dists = [
    [83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
    [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],
    [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
    [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
    [51, 114, 72, 54, 69, 139, 105, 155, 62],
    [70, 25, 22, 52, 90, 56, 105, 16],
    [45, 61, 111, 36, 61, 57, 70],
    [23, 71, 67, 48, 85, 29],
    [74, 89, 69, 107, 36],
    [117, 65, 125, 43],
    [54, 22, 84],
    [60, 44],
    [97],
    [],
]

## Creating n and V vars

In [18]:
n, V = len(dists), set(range(len(dists)))

## Creating $C_{ij}$

In [19]:
c = [
    [0 if i == j else dists[i][j - i - 1] if j > i else dists[j][i - j - 1] for j in V]
    for i in V
]

In [20]:
model = Model()

## Creating the decision variables

In [21]:
x = [[model.add_var(var_type=BINARY) for _ in V] for _ in V]

## y var for subtour elimination

In [22]:
y = [model.add_var() for _ in V]

## Objective function

In [23]:
model.objective = minimize(xsum(c[i][j] * x[i][j] for i in V for j in V))

## Constraints

In [24]:
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

In [25]:
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

In [26]:
for i, j in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n + 1) * x[i][j] >= y[j] - n

In [27]:
model.optimize()

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 184 (0) rows, 195 (-15) columns and 832 (0) elements
Clp1000I sum of infeasibilities 8.22201e-05 - average 4.46848e-07, 124 fixed columns
Coin0506I Presolve 182 (-2) rows, 69 (-126) columns and 474 (-358) elements
Clp0006I 0  Obj 399.20349 Primal inf 3.3278783e-05 (22) Dual inf 2.1855024e+13 (56)
Clp0029I End of values pass after 69 iterations
Clp0014I Perturbing problem by 0.001% of 5.362616 - largest nonzero change 3.4125692e-05 ( 0.00073948616%) - largest zero change 1.8240846e-05
Clp0000I Optimal - objective value 399.2
Clp0000I Optimal - objective value 399.2
Coin0511I After Postsolve, objective 399.2, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 399.2
Clp0000I Optimal - objective value 399.2
Clp0000I Optimal - objective value 399.2
Clp0000I Optimal - objective value 399.2
Coin0511I After Postsolve, objective 399.2, infeasibilities - dual 0 (0), primal 0 (0)
Clp003

<OptimizationStatus.OPTIMAL: 0>

In [28]:
if model.num_solutions:
    print(f"Route with total distance: {model.objective_value} found: {places[0]}")
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        print(f"-> {places[nc]}")
        if nc == 0:
            break

Route with total distance: 547.0 found: Antwerp
-> Mechelen
-> Leuven
-> Hasselt
-> C-Mine
-> Montagne de Bueren
-> Remouchamps
-> Dinant
-> Namur
-> Mons
-> Waterloo
-> Grand-Place de Bruxelles
-> Ghent
-> Bruges
-> Antwerp
